In [1]:
import os

In [2]:
%pwd

'/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

### Config.yaml (just save not for run)

In [ ]:
artifacts_root: artifacts


data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://drive.google.com/file/d/1q5C4ct4TycP0cAdkcK1KXr8VsId-fE1k/view?usp=sharing  
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

### Prams.yaml (just save not for run)

In [ ]:
key: value

### Code for entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### configuration manager in src confiq

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import DataIngestionConfig

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

### data ingestion component

In [19]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
from cnnClassifier.entity.config_entity import DataIngestionConfig

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




### Pipeline - 01

##### run this code for trails this node book

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-16 18:28:13,304: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-16 18:28:13,306: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-16 18:28:13,307: INFO: common: created directory at: artifacts]
[2025-04-16 18:28:13,308: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-16 18:28:13,309: INFO: 586056171: Downloading data from https://drive.google.com/file/d/1q5C4ct4TycP0cAdkcK1KXr8VsId-fE1k/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1q5C4ct4TycP0cAdkcK1KXr8VsId-fE1k
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1q5C4ct4TycP0cAdkcK1KXr8VsId-fE1k&confirm=t&uuid=95f7cfae-4584-4e11-94bf-f6ce70291010
To: /Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 49.2M/49.2M [09:21<00:00, 87.6kB/s]

[2025-04-16 18:37:42,648: INFO: 586056171: Downloaded data from https://drive.google.com/file/d/1q5C4ct4TycP0cAdkcK1KXr8VsId-fE1k/view?usp=sharing into file artifacts/data_ingestion/data.zip]


###### This code for modular coddig

In [ ]:
### this code for modular coddig
from cnnClassifier.config.configuration import ConfigurationManager
from cnnClassifier.components.data_ingestion import DataIngestion
from cnnClassifier import logger

### this code for modular coddig

STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()




if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e